# Семинар 4 – Линейные модели в задачах классификации

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.stats as st
import warnings
warnings.simplefilter('ignore')

# 1. Постановка задачи

![picture](https://drive.google.com/uc?export=view&id=1Pswe7OKfk75MvJa6kmIRJn6Dq4LhrPTX)

Где линейная модель - это: $$ \hat{y} = f(x) = \theta_0*1 + \theta_1*x_1 + ... + \theta_n*x_n = \theta^T*X$$

# 1. Метрики

In [ ]:
from sklearn.datasets import fetch_olivetti_faces

In [ ]:
data = fetch_olivetti_faces()
data.target

In [ ]:
X, y = data.data, data.target

In [ ]:
n_row, n_col = 2, 3
n_components = n_row * n_col
image_shape = (64, 64)
rng = 0

# #############################################################################
# Load faces data
faces, _ = fetch_olivetti_faces(return_X_y=True, shuffle=True,
                                random_state=rng)
n_samples, n_features = faces.shape

def plot_gallery(title, images, n_col=n_col, n_row=n_row, cmap=plt.cm.gray):
    plt.figure(figsize=(2. * n_col, 2.26 * n_row))
    plt.suptitle(title, size=16)
    for i, comp in enumerate(images):
        plt.subplot(n_row, n_col, i + 1)
        vmax = max(comp.max(), -comp.min())
        plt.imshow(comp.reshape(image_shape), cmap=cmap,
                   interpolation='nearest',
                   vmin=-vmax, vmax=vmax)
        plt.xticks(())
        plt.yticks(())
    plt.subplots_adjust(0.01, 0.05, 0.99, 0.93, 0.04, 0.)

In [ ]:
plot_gallery("Olivetti faces", faces[:n_components])

### Подготовим заведомо несбалансированную выборку (3 класса)

In [ ]:
indx_0 = np.where(y == 0)[0]
indx_1 = np.where(y == 1)[0][:8]
print(indx_0)
print(indx_1)

In [ ]:
indx_2 = np.where(y == 2)[0][:3]
indx_2

In [ ]:
X = X[np.concatenate((np.concatenate((indx_1, indx_0)), indx_2))]
y = y[np.concatenate((np.concatenate((indx_1, indx_0)), indx_2))]

In [ ]:
print('В датасете {} объектов и {} признака'.format(X.shape[0], X.shape[1]))

## Разделим выборку на две части: обучающую и тестовую

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.5,
                                                    test_size=0.5,
                                                    shuffle=True,
                                                   random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

Зададим классификатор:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
knn.fit(X_train, y_train)
knn_predictons = knn.predict(X_test)

In [ ]:
preds = pd.DataFrame(y_test, columns=['True'])
preds['knn_pred'] = knn_predictons

In [ ]:
preds

Что можно заметить?

---

На 0 и 1 класса классификатор отработал хорошо, но 2 класс он не видел и не смог его определить

## Посмотрим долю правильных ответов:

In [ ]:
def accuracy(true, predictions):
    acc = predictions[true == predictions].shape[0]/true.shape[0] # Ваш код здесь
    return acc
accuracy(y_test, knn_predictons)

In [ ]:
#Тоже самое средставми sklearn:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, knn_predictons)

## Посмотрим точность ответов (precision)
без учета различных классов (всего TP/ (всего TP + всего FP)):


In [ ]:
def precision(true, predictions):
    # Ваш код здесь:
    # Выбираем класс 1 как True
    TP = 0
    FP = 0
    for i in range(len(true)):
      if true[i] == 1 and predictions[i] == 1:
        TP = TP + 1
      elif predictions [i] != true[i] and predictions[i] == 1:
        FP = FP + 1
    prec = TP/(TP+FP)
    return prec
precision(y_test, knn_predictons)

Тоже самое средставми sklearn:

---

'binary':
 для класса, указанного pos_label. Это применимо, только если Таргет (y_ {true, pred}) является двоичным.

'micro':
подсчитывает общее количество TP и FP.

'macro':
для каждого класса и берет их невзвешенное среднее. При этом не учитывается дисбаланс классов.

'weighted':
для каждого класса подсчитывается TP и FP отдельно и берется их средневзвешенное значение

'samples':
для каждого экземпляра и берется их среднее значение

In [ ]:
from sklearn.metrics import precision_score
precision_score(y_test, knn_predictons, labels=[1], average='micro')

## Посмотрим полноту ответов (recall)
без учета различных классов (всего TP/(всего TP + всего FN)):

In [ ]:
def recall(true, predictions):
    # Ваш код здесь:
    TP = 0
    FN = 0
    for i in range(len(true)):
      if true[i] == 1 and predictions[i] == 1:
        TP = TP + 1
      elif predictions [i] != 1 and true[i] == 1:
        FN = FN + 1
    rec = TP/(TP+FN)
    return rec
recall(y_test, knn_predictons)

Тоже самое средставми sklearn:

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test, knn_predictons, labels=[1], average='micro')

## Посмотрим F-score
2 * precision * recall / (precision + recall)

In [ ]:
def F1_score(true, predictions):
    # Ваш код здесь:
    f1 = 2*(recall(true, predictions)*precision(true, predictions))/(recall(true, predictions)+precision(true, predictions))
    return f1
F1_score(y_test, knn_predictons)

Тоже самое средставми sklearn:

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, knn_predictons, labels=[1], average='micro')

## Теперь давайте построим ROC curve:

Для этого, в отличае от других метрий, нам понадобятся не предсказания модели, а вероятности принадлежности классам, которые модель выдает.

In [ ]:
probs = knn.predict_proba(X_test)

Также необходибо бинаризовать метки классов для подсчета TPR и FPR

In [ ]:
from sklearn.preprocessing import label_binarize
counts = np.unique(y, return_counts=True)
y_test_bin = label_binarize(y_test, classes=counts[0])

In [ ]:
y_test_bin[0]

Считаем TPR и FPR для каждого класса

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in counts[0]:
    fpr[i], tpr[i], threshold = roc_curve(y_test_bin[:, i], probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
roc_auc

In [ ]:
fpr["micro"], tpr["micro"], threshold = roc_curve(y_test_bin.ravel(), probs.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
roc_auc["micro"]

In [ ]:
def plot_roc_curve(index=8):
    plt.figure()
    lw = 2
    plt.plot(fpr[index], tpr[index], color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[index])
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic for class {}'.format(index))
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
plot_roc_curve("micro")

# Линейные модели для задачи классификации
В качестве демонстрации будем использовальзовать датасет с изображениями цифр

In [ ]:
# Загрузим данные
from sklearn.datasets import load_digits
digits = load_digits()

In [ ]:
X = digits.data
y = digits.target

In [ ]:
# Посмотрим на кол-во объектов
X.shape, y.shape

In [ ]:
# Давайте, посмотрим, как вылядит случайный объект нашей выборки
target_image_id = np.random.randint(X.shape[0])

plt.figure(figsize=(4,2))
plt.imshow(X[target_image_id].reshape((8, 8)), cmap='binary')
plt.xticks([])
plt.yticks([])
plt.title('True class: '+ str(y[target_image_id]))
plt.show()

In [ ]:
# Посмотрим на баланс классов
class_counts = np.unique(y, return_counts=True)

pd.DataFrame(class_counts[1], index=class_counts[0], columns=['Counts'])

## Логистическая регрессия для многоклассовой классификации

In [ ]:
# Загрузим нужные библиотеки
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [ ]:
# Разделим выборку на тренировочную и тестовую
x_train, x_test, y_train, y_test = train_test_split(X, y,
                 train_size=0.8, test_size=0.2, shuffle=True,
                 random_state=42)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

*Логистическая регрессия позволяет решать задачу многоклассовой классификации. Класс ``LogisticRegression`` позвляет это делать двумя способами:*
- Стандартный One vs Rest (т.е. каждый класс отделяется от всех других). Параметр `multi_class='ovr'`.*
- One vs One: Используя кросс-энтропию (оценивается сразу вектор вероятностей принадлежности классам). Параметр `multi_class='multinomial'`.*

#### One vs Rest


Find 𝐾 − 1 classifiers 𝑓 , 𝑓 , ... , 𝑓 12 𝐾−1  
- 𝑓 classifies1𝑣𝑠{2,3,...,𝐾} 1
- 𝑓 classifies2𝑣𝑠{1,3,...,𝐾} 2
- ...
- 𝑓 classifies𝐾−1𝑣𝑠{1,2,...,𝐾−2}
- 𝐾−1
- Points not classified to classes {1,2, ... , 𝐾 − 1} are put to class 𝐾


#### One vs One (Cross-entropy)


В случае с бинарной классификацией функция потерь:
$$ \sum_{i=1}^l \bigl( y_i \log a_i - (1-y_i) \log (1-a_i) \bigr)  \rightarrow min$$  
$a_i$ – ответ (вероятность) алгоритма на i-м объекте на вопрос принадлежности к классу $y_i$

Обобщается для многомерного случая:
$$-\frac{1}{q} \sum_{i=1}^q \sum_{j=1}^l y_{ij} \log a_{ij} \rightarrow min $$
где  
$q$ – число элементов в выборке,  
$l$ – число классов,   
$a_{ij}$ – ответ (вероятность) алгоритма на i-м объекте на вопрос принадлежности его к j-му классу

__Проблемы:__

- Сложности в поиске глобального минимума, так как присутствуют Локальные минимумы и плато

## Solvers

![picture](https://drive.google.com/uc?export=view&id=19r1jQUiEStMMrGJCWAxzgjFiG4bvXIZ3)

Source: [User Guide](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)

### Liblinear
Используется обычный покоординантный спуск.
Алгоритм:
- Инициализацируем любыми значениями вектор весов
- Повторяем для каждого i из пространства признаков:
    - фиксируем значения всех переменных кроме $x_i$
    - проводим одномерную оптимизацию по переменной $x_i$, любым методом одномерной оптимизации
    - если достигли минимума по одной координате, то возвращаем текущее значение вектора весов

Как это выглядит для минимизации функционала

![picture](https://drive.google.com/uc?export=view&id=13J7wRDNmNNeueuT9rciTl2Quw60UYIbv)

__Недостатки:__
1. Не параллелится
2. Может "застрять" в локальном минимуме
3. Следствие п.2 - Не может использоваться кросс-энтропия для многомерного случая, так как легко "Застревает" в локальных минимумах. Вместо этого для каждого класса строит отдельный классификатор (One-vs-Rest)

In [ ]:
%%time
lr = LogisticRegression(solver='liblinear', multi_class='ovr')
lr.fit(x_train, y_train)

In [ ]:
accuracy_score(lr.predict(x_test), y_test)

In [ ]:
%%time
len_c = 10
param_grid={
    'C': np.linspace(0.01, 1, len_c),
    'penalty': ['l1', 'l2']
    }

gs=GridSearchCV(lr,param_grid=param_grid, cv=3,
                n_jobs=-1, scoring='accuracy')
gs.fit(x_train, y_train)

In [ ]:
accuracy_score(gs.predict(x_test), y_test)

In [ ]:
def print_cv_results(a, len_gs, params, param_r, param_sep):
    d = len(params['param_grid'][param_sep])
    ar=np.array(a).reshape(d, len_gs).T

    df=pd.DataFrame(ar)

    pen_par=params['param_grid'][param_sep]
    c_par=params['param_grid'][param_r].tolist()
    columns_mapper=dict(zip(range(0, len(pen_par)),pen_par))
    row_mapper=dict(zip(range(0, len(c_par)), c_par))

    df.rename(columns=columns_mapper, index=row_mapper, inplace=True)

    plot = df.plot(title='Mean accuracy rating',grid=True)
    plot.set_xlabel(param_r, fontsize=13)
    plot.set_ylabel('acc', rotation=0, fontsize=13, labelpad=15)

In [ ]:
print_cv_results(gs.cv_results_['mean_test_score'],
                 len_c, gs.get_params(), 'C','penalty')

### Newton-cg (Newton’s Method)

Геометрическая интерпретация метода Ньютона заключается в том, что на каждой итерации приближается f(x) квадратичной функцией, а затем делает шаг к максимуму / минимуму этой квадратичной функции.

Недостатки:

1. Затратно с точки зрения вычислений матрицы Гессе (т.е. вычислений вторых частных производных).

2. Может остановиться в седловой точке, которые часто появляются при многопараметрической оптимизации.

In [ ]:
%%time
lr = LogisticRegression(solver='newton-cg', penalty='l2')
lr.fit(x_train, y_train)

In [ ]:
accuracy_score(lr.predict(x_test), y_test)

In [ ]:
%%time
len_c = 10
param_grid={
    'C': np.linspace(0.01, 1, len_c),
    'multi_class': ['ovr', 'multinomial']
    }

gs=GridSearchCV(lr,param_grid=param_grid, cv=3,
                n_jobs=-1, scoring='accuracy')
gs.fit(x_train, y_train)

In [ ]:
accuracy_score(gs.predict(x_test), y_test)

In [ ]:
print_cv_results(gs.cv_results_['mean_test_score'],
                 len_c, gs.get_params(), 'C','multi_class')

### Lbfgs (Limited-memory Broyden–Fletcher–Goldfarb–Shanno Algorithm)

Аналог метода Ньютона (квазиньютоновский метод), но здесь матрица Гессе аппроксимируется с использованием оценок градиента.

Ограниченная память означает, что хранится только несколько векторов, которые неявно представляют приближение матрицы.

Solver «lbfgs» рекомендуется использовать для небольших наборов данных, так как для больших наборов данных снижается его производительность.

“lbfgs” solver используется в sklearn по умолчанию из-за его устойчивости.

In [ ]:
%%time
lr = LogisticRegression(solver='lbfgs', penalty='l2')
lr.fit(x_train, y_train)

In [ ]:
accuracy_score(lr.predict(x_test), y_test)

In [ ]:
%%time
len_c = 10
param_grid={
    'C': np.linspace(0.01, 1, len_c),
    'multi_class': ['ovr', 'multinomial']
    }

gs=GridSearchCV(lr,param_grid=param_grid, cv=3,
                n_jobs=-1, scoring='accuracy')
gs.fit(x_train, y_train)

In [ ]:
accuracy_score(gs.predict(x_test), y_test)

In [ ]:
print_cv_results(gs.cv_results_['mean_test_score'],
                 len_c, gs.get_params(), 'C','multi_class')

### Stochatic Average Gradient (SAG)

Объединение градиентного спуска и стохастического.
При этом, он имеет низкую стоимость итерации свойственной SGD, но делает шаг градиента по отношению к аппроксимации полного градиента:


__Недостатки:__
- Нет L1
- Непрактичен для больших выборок, так как имеет высокую вычислительную сложность

In [ ]:
%%time
lr = LogisticRegression(solver='sag', penalty='l2')
lr.fit(x_train, y_train)

In [ ]:
accuracy_score(lr.predict(x_test), y_test)

In [ ]:
%%time
len_c = 10
param_grid={
    'C': np.linspace(0.01, 1, len_c),
    'multi_class': ['ovr', 'multinomial']
    }

gs=GridSearchCV(lr,param_grid=param_grid, cv=3,
                n_jobs=-1, scoring='accuracy')
gs.fit(x_train, y_train)

In [ ]:
accuracy_score(gs.predict(x_test), y_test)

In [ ]:
print_cv_results(gs.cv_results_['mean_test_score'],
                 len_c, gs.get_params(), 'C','multi_class')

### Stochatic Average Gradient Augmented (SAGA)

SAGA является вариантом SAG, но который поддерживает опцию non-smooth penalty=l1 (т. е. регуляризацию L1).

Кроме того, это единственный Solver, поддерживающий регуляризацию = "elasticnet".

[Подробнее: ](https://www.di.ens.fr/~fbach/Defazio_NIPS2014.pdf)

In [ ]:
lr_clf = LogisticRegression(solver='saga', max_iter=1500)

In [ ]:
%%time
len_c = 3
param_grid={
    'C': np.linspace(0.01, 1, len_c),
    #'multi_class': ['ovr', 'multinomial'],
    'penalty': ['l1', 'l2']
    }

gs=GridSearchCV(lr_clf,param_grid=param_grid, cv=3,
                n_jobs=-1, scoring='accuracy')
gs.fit(x_train, y_train)

In [ ]:
print_cv_results(gs.cv_results_['mean_test_score'],
                 len_c, gs.get_params(), 'C','penalty')

In [ ]:
accuracy_score(gs.predict(x_test), y_test)

# Intro to PyTorch (по материалам [DLсourse](https://www.dlschool.org/))

[official PyTorch website](https://pytorch.org/).

## Syntax

In [ ]:
import torch

Some facts about PyTorch:  
- dynamic computation graph
- handy `torch.nn` and `torchvision` modules for fast neural network prototyping
- even faster than TensorFlow on some tasks
- allows to use GPU easily

If PyTorch was a formula, it would be:  

$$PyTorch = NumPy + CUDA + Autograd$$

(CUDA - [wiki](https://en.wikipedia.org/wiki/CUDA))

Let's see how we can use PyTorch to operate with vectors and tensors.  

Recall that **a tensor** is a multidimensional vector, e.g. :  

`x = np.array([1,2,3])` -- a vector = a tensor with 1 dimension (to be more precise: `(1,)`)  
`y = np.array([[1, 2, 3], [4, 5, 6]])` -- a matrix = a tensor with 2 dimensions (`(2, 3)` in this case)  
`z = np.array([[[1, 2, 3], [4, 5, 6], [7, 8, 9]],  
               [[1, 2, 3], [4, 5, 6], [7, 8, 9]],  
               [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])` -- "a cube" (3, 3, 3) = a tensor with 3 dimensions (`(3, 3, 3)` in this case)

One real example of 3-dimensional tensor is **an image**, it has 3 dimensions: `height`, `width` and the `channel depth` (= 3 for color images, 1 for a greyscale). You can think of it as of parallelepiped consisting of the real numbers.

In PyTorch we will use `torch.Tensor` (`FloatTensor`, `IntTensor`, `ByteTensor`) for all the computations.

All tensor types:

In [ ]:
torch.HalfTensor      # 16 бит, floating point
torch.FloatTensor     # 32 бита, floating point
torch.DoubleTensor    # 64 бита, floating point

torch.ShortTensor     # 16 бит, integer, signed
torch.IntTensor       # 32 бита, integer, signed
torch.LongTensor      # 64 бита, integer, signed

torch.CharTensor      # 8 бит, integer, signed
torch.ByteTensor      # 8 бит, integer, unsigned

We will use only `torch.FloatTensor()` and `torch.IntTensor()`.

Let's begin to do something!

* Creating the tensor:

In [ ]:
a = torch.FloatTensor([1, 2])
a


In [ ]:
a.shape

In [ ]:
b = torch.FloatTensor([[1,2,3], [4,5,6]])
b

In [ ]:
b.shape

In [ ]:
x = torch.FloatTensor(2,3,4)

In [ ]:
x

In [ ]:
x = torch.FloatTensor(100)
x

In [ ]:
x = torch.IntTensor(45, 57, 14, 2)
x.shape

**Note:** if you create `torch.Tensor` with the following constructor it will be filled with the "random trash numbers":

In [ ]:
x = torch.IntTensor(3, 2, 4)
x

Here is a way to fill a new tensor with zeroes:

In [ ]:
x = torch.FloatTensor(3, 2, 4).zero_()
x

## Numpy -> Torch

All numpy function have its pair in torch.

https://github.com/torch/torch7/wiki/Torch-for-Numpy-users

`np.reshape()` == `torch.view()`:

In [ ]:
b

In [ ]:
b.view(3, 2)

**Note:** `torch.view()` creates a new tensor, one the old one remains unchanged

In [ ]:
b.view(-1)

In [ ]:
b

* Change a tensor type:

In [ ]:
a = torch.FloatTensor([1.5, 3.2, -7])

In [ ]:
a.type_as(torch.IntTensor())

In [ ]:
a.type_as(torch.ByteTensor())

**Note:** `.type_as()` creates a new tensor, the old one remains unchanged

In [ ]:
a

* Indexing is just like in `NumPy`:

In [ ]:
a = torch.FloatTensor([[100, 20, 35], [15, 163, 534], [52, 90, 66]])
a

In [ ]:
a[0, 0]

In [ ]:
a[0:2, 0:2]

**Ariphmetics and boolean operations** and their analogues:  

| Operator | Analogue |
|:-:|:-:|
|`+`| `torch.add()` |
|`-`| `torch.sub()` |
|`*`| `torch.mul()` |
|`/`| `torch.div()` |

* Addition:

In [ ]:
a = torch.FloatTensor([[1, 2, 3], [10, 20, 30], [100, 200, 300]])
b = torch.FloatTensor([[-1, -2, -3], [-10, -20, -30], [100, 200, 300]])

In [ ]:
a + b

In [ ]:
a.add(b)

In [ ]:
b = -a
b

In [ ]:
a + b

* Subtraction:

In [ ]:
a - b

In [ ]:
a.sub(b)

* Multiplication (elementwise):

In [ ]:
a * b

In [ ]:
a.mul(b)

* Division (elementwise):

In [ ]:
a = torch.FloatTensor([[1, 2, 3], [10, 20, 30], [100, 200, 300]])
b = torch.FloatTensor([[-1, -2, -3], [-10, -20, -30], [100, 200, 300]])

In [ ]:
a / b

In [ ]:
a.div(b)

**Note:** all this operations create new tensors, the old tensors remain unchanged

In [ ]:
a

In [ ]:
b

* Comparison operators:

In [ ]:
a = torch.FloatTensor([[1, 2, 3], [10, 20, 30], [100, 200, 300]])
b = torch.FloatTensor([[-1, -2, -3], [-10, -20, -30], [100, 200, 300]])

In [ ]:
a == b

In [ ]:
a != b

In [ ]:
a < b

In [ ]:
a > b

* Using boolean mask indexing:

In [ ]:
a[a > b]

In [ ]:
b[a == b]

Elementwise application of the **universal functions**:

In [ ]:
a = torch.FloatTensor([[1, 2, 3], [10, 20, 30], [100, 200, 300]])

In [ ]:
a.sin()

In [ ]:
torch.sin(a)

In [ ]:
a.tan()

In [ ]:
a.exp()

In [ ]:
a.log()

In [ ]:
b = -a
b

In [ ]:
b.abs()

* The sum, mean, max, min:

In [ ]:
a.sum()

In [ ]:
a.mean()

Along axis:

In [ ]:
a

In [ ]:
a.sum(dim=0)

In [ ]:
a.sum(1)

In [ ]:
a.max()

In [ ]:
a.max(0)

In [ ]:
a.min()

In [ ]:
a.min(0)

**Note:** the second tensor returned by `.max()` and `.min()` contains the indices of max/min elements along this axis. E.g. in that case `a.min()` returned `(1, 2, 3)` which are the minimum elements along 0 axis (along columns) and their indices along 0 axis are `(0, 0, 0)`.

## Matrix operations

* Transpose a tensor:

In [ ]:
a = torch.FloatTensor([[1, 2, 3], [10, 20, 30], [100, 200, 300]])
a

In [ ]:
a.t()

It is not not the inplace operation too:

In [ ]:
a

* Dot product of vectors:

In [ ]:
a = torch.FloatTensor([1, 2, 3, 4, 5, 6])
b = torch.FloatTensor([-1, -2, -4, -6, -8, -10])

In [ ]:
a.dot(b)

In [ ]:
a.shape, b.shape

In [ ]:
a @ b

In [ ]:
type(a)

In [ ]:
type(b)

In [ ]:
type(a @ b)

* Matrix product:

In [ ]:
a = torch.FloatTensor([[1, 2, 3], [10, 20, 30], [100, 200, 300]])
b = torch.FloatTensor([[-1, -2, -3], [-10, -20, -30], [100, 200, 300]])

In [ ]:
a.mm(b)

In [ ]:
a @ b

Remain unchanged:

In [ ]:
a

In [ ]:
b

In [ ]:
a = torch.FloatTensor([[1, 2, 3], [10, 20, 30], [100, 200, 300]])
b = torch.FloatTensor([[-1], [-10], [100]])

In [ ]:
print(a.shape, b.shape)

In [ ]:
a @ b

If we unroll the tensor `b` in an array (`torch.view(-1)`) the multiplication would be like with the column:

In [ ]:
b

In [ ]:
b.view(-1)

In [ ]:
a @ b.view(-1)

In [ ]:
a.mv(b.view(-1))

## From NumPy to PyTorch conversion

In [ ]:
import numpy as np

a = np.random.rand(3, 3)
a

In [ ]:
b = torch.from_numpy(a)
b

**NOTE!** `a` and `b` have the same data storage, so the changes in one tensor will lead to the changes in another:

In [ ]:
b -= b
b

In [ ]:
a

**From PyTorch to NumPy conversion:**

In [ ]:
a = torch.FloatTensor(2, 3, 4)
a

In [ ]:
type(a)

In [ ]:
x = a.numpy()
x

In [ ]:
x.shape

In [ ]:
type(x)

In [ ]:
x -= x

In [ ]:
a

Let's write the `forward_pass(X, w)` ($w_0$ is a part of the $w$) for a single neuron (activation = sigmoid) using PyTorch:

In [ ]:
# Ваш код
def forward_pass(X, w):
    return torch.sigmoid(X @ w)

In [ ]:
X = torch.FloatTensor([[-5, 5],
                       [2, 3],
                       [1, -1]])

w = torch.FloatTensor([[-0.5],
                       [2.5]])

result = forward_pass(X, w)
print('result: {}'.format(result))

## CUDA

[CUDA documentation](https://docs.nvidia.com/cuda/)

We can use both CPU (Central Processing Unit) and GPU (Graphical Processing Unit) to make the computations with PyTorch. We can switch between them easily, this is one of the most important things in PyTorch framework.

In [ ]:
x = torch.FloatTensor(1024, 1024).uniform_()
x

In [ ]:
x.is_cuda

Place a tensor on GPU:

In [ ]:
x = x.cuda()

In [ ]:
x

In [ ]:
device = torch.device("cuda:0")
x = x.to(device)
x

Let's multiply two tensors on GPU and then move the result on the CPU:

In [ ]:
a = torch.FloatTensor(10000, 10000).uniform_()
b = torch.FloatTensor(10000, 10000).uniform_()
c = a.cuda().mul(b.cuda()).cpu()

In [ ]:
c

In [ ]:
a

Tensors placed on CPU and tensors placed on GPU are unavailable for each other:

In [ ]:
a = torch.FloatTensor(10000, 10000).uniform_().cpu()
b = torch.FloatTensor(10000, 10000).uniform_().cuda()

In [ ]:
a + b

Example of working with GPU:

In [ ]:
x = torch.FloatTensor(5, 5, 5).uniform_()

# check for CUDA availability (NVIDIA GPU)
if torch.cuda.is_available():
    # get the CUDA device name
    device = torch.device('cuda')          # CUDA-device object
    y = torch.ones_like(x, device=device)  # create a tensor on GPU
    x = x.to(device)                       # or just `.to("cuda")`
    z = x + y
    print(z)
    # you can set the type while `.to` operation
    print(z.to("cpu", torch.double))

## Autograd

The autograd package provides automatic differentiation for all operations on Tensors. It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

The examples:

In [ ]:
dtype = torch.float
device = torch.device("cuda:0")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 3, 3, 10

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.

# Ваш код здесь

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.

# Ваш код здесь


# Посчитаем квадратичный лосс
# Ваш код здесь

print(loss)
# calculate the gradients
loss.backward()

In [ ]:
loss.grad

In [ ]:
w1.grad

In [ ]:
x.grad

In [ ]:
y.grad

**NOTE:** the gradients are placed into the `.grad` field of tensors (variables) on which gradients were calculated. Gradients *are not placed* in the variable `loss` here!

In [ ]:
w1

## Implementation

Any self-respecting DL framework must do your backprop for you. Torch handles this with the `autograd` module.

The general pipeline looks like this:
* When creating a tensor, you mark it as `requires_grad`:
    * __```torch.zeros(5, requires_grad=True)```__
    * torch.tensor(np.arange(5), dtype=torch.float32, requires_grad=True)
* Define some differentiable `loss = arbitrary_function(a)`
* Call `loss.backward()`
* Gradients are now available as ```a.grads```

__Here's an example:__ let's fit a linear regression on Boston house prices

In [ ]:
#!pip install scikit-learn==1.1.3

In [ ]:
# import pandas as pd
# import numpy as np
# %matplotlib inline
# import matplotlib.pyplot as plt
# import torch

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()
plt.scatter(boston.data[:, -1], boston.target)

If you compute gradient from multiple losses, the gradients will add up at variables, therefore it's useful to __zero the gradients__ between iteratons.

In [ ]:
from IPython.display import clear_output
lr = 0.05

# Инициализируйте переменные




for i in range(100):

    # Предсказание (Ваш код)


    # loss (Ваш код)


    # обновление весов (Ваш код)


    # обнуление градиента (Ваш код)


    # the rest of code is just bells and whistles
    if (i+1)%5==0:
        clear_output(True)
        plt.scatter(x.data.numpy(), y.data.numpy())
        plt.scatter(x.data.numpy(), y_pred.data.numpy(), color='orange', linewidth=5)
        plt.show()

        print("loss = ", loss.data.numpy())
        if loss.data.numpy() < 0.1:
            print("Done!")
            break


Suprizingly, we were walking really close to the edge. Look a few cells above. We have divided the `x` values by 10 times. Let's what happens if we don't:

In [ ]:
plt.plot([element[0] for element in grad_history], )

In [ ]:
print(grad_history)

# High-level pytorch

So far we've been dealing with low-level torch API. While it's absolutely vital for any custom losses or layers, building large neura nets in it is a bit clumsy.

Luckily, there's also a high-level torch interface with a pre-defined layers, activations and training algorithms.

We'll cover them as we go through a simple image recognition problem


In [ ]:
from sklearn.datasets import load_digits
dataset = load_digits()

features = dataset.data
target = dataset.target

features.shape, target.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25)

print(X_train.shape, y_train.shape)

In [ ]:
binary_train_mask = (y_train == 0) | (y_train == 1)
X_train = X_train[binary_train_mask]
y_train = y_train[binary_train_mask]

binary_test_mask = (y_test == 0) | (y_test == 1)
X_test = X_test[binary_test_mask]
y_test = y_test[binary_test_mask]

In [ ]:
y_test

In [ ]:
for i in [0,1]:
    plt.subplot(1, 2, i + 1)
    plt.imshow(X_train[i].reshape([8,8]))
    plt.title(str(y_train[i]))

Let's start with layers. The main abstraction here is __`torch.nn.Module`__

In [ ]:
from torch import nn
import torch.nn.functional as F

print(nn.Module.__doc__)

There's a vast library of popular layers and architectures already built for ya'.

This is a binary classification problem, so we'll train a __Logistic Regression with sigmoid__.
$$P(y_i | X_i) = \sigma(W \cdot X_i + b) ={ 1 \over {1+e^{- [W \cdot X_i + b]}} }$$


In [ ]:
# create a network that stacks layers on top of each other
model = nn.Sequential()

# add first "dense" layer with 64 input units and 1 output unit.
model.add_module('l1', nn.Linear(64, 1))

# add softmax activation for probabilities. Normalize over axis 1
# note: layer names must be unique
model.add_module('l2', nn.Sigmoid())

In [ ]:
print("Weight shapes:", [w.shape for w in model.parameters()])

In [ ]:
# create dummy data with 3 samples and 64 features
x = torch.tensor(X_train[:3], dtype=torch.float32)
y = torch.tensor(y_train[:3], dtype=torch.float32)

# compute outputs given inputs, both are variables
y_predicted = model(x)[:, 0]

y_predicted # display what we've got

Let's now define a loss function for our model.

The natural choice is to use binary crossentropy (aka logloss, negative llh):
$$ L = {1 \over N} \underset{X_i,y_i} \sum - [  y_i \cdot log P(y_i | X_i) + (1-y_i) \cdot log (1-P(y_i | X_i)) ]$$



In [ ]:
F.binary_cross_entropy

In [ ]:
loss = F.binary_cross_entropy(y_predicted, y)

In [ ]:
loss

__Note:__ you can also find many such functions in `torch.nn.functional`, just type __`F.<tab>`__.

__Torch optimizers__

When we trained Linear Regression above, we had to manually .zero_() gradients on both our variables. Imagine that code for a 50-layer network.

Again, to keep it from getting dirty, there's `torch.optim` module with pre-implemented algorithms:

In [ ]:
opt = torch.optim.SGD(model.parameters(), lr=0.01)

# here's how it's used:
loss.backward()      # add new gradients
opt.step()           # change weights
opt.zero_grad()      # clear gradients

In [ ]:
# dispose of old variables to avoid bugs later
del x, y, y_predicted, loss

### Putting it all together

In [ ]:
from torch import nn
import torch.nn.functional as F

# create network again just in case
model = nn.Sequential()
model.add_module('first', nn.Linear(64, 1))
model.add_module('second', nn.Sigmoid())


opt = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
history = []

for i in range(1000):

    # sample 256 random images


    # predict probabilities


    assert y_predicted.dim() == 1, "did you forget to select first column with [:, 0]"

    # compute loss, just like before


    # compute gradients


    # SGD step



    # clear gradients



    history.append(loss.data.numpy())

    if i % 10 == 0:
        print("step #%i | mean loss = %.3f" % (i, np.mean(history[-10:])))

__Debugging tips:__
* make sure your model predicts probabilities correctly. Just print them and see what's inside.
* don't forget _minus_ sign in the loss function! It's a mistake 99% ppl do at some point.
* make sure you zero-out gradients after each step. Srsly:)
* In general, pytorch's error messages are quite helpful, read 'em before you google 'em.
* if you see nan/inf, print what happens at each iteration to find our where exactly it occurs.
  * If loss goes down and then turns nan midway through, try smaller learning rate. (Our current loss formula is unstable).


### Evaluation

Let's see how our model performs on test data

In [ ]:
X_test

In [ ]:
# use your model to predict classes (0 or 1) for all test samples
predicted_y_test = model(torch.tensor(X_test, dtype=torch.float32))[:, 0]
predicted_y_test = np.array(predicted_y_test > 0.5)

assert isinstance(predicted_y_test, np.ndarray), "please return np array, not %s" % type(predicted_y_test)
assert predicted_y_test.shape == y_test.shape, "please predict one class for each test sample"
assert np.in1d(predicted_y_test, y_test).all(), "please predict class indexes"

accuracy = np.mean(predicted_y_test == y_test)

print("Test accuracy: %.5f" % accuracy)
assert accuracy > 0.95, "try training longer"

print('Great job!')

### More about pytorch:
* Using torch on GPU and multi-GPU - [link](http://pytorch.org/docs/master/notes/cuda.html)
* More tutorials on pytorch - [link](http://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)
* Pytorch examples - a repo that implements many cool DL models in pytorch - [link](https://github.com/pytorch/examples)
* Practical pytorch - a repo that implements some... other cool DL models... yes, in pytorch - [link](https://github.com/spro/practical-pytorch)
* And some more - [link](https://www.reddit.com/r/pytorch/comments/6z0yeo/pytorch_and_pytorch_tricks_for_kaggle/)

# Ссылки

*1). Official PyTorch tutorials: https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py*

*2). arXiv article about the deep learning frameworks comparison: https://arxiv.org/pdf/1511.06435.pdf*

*3). Useful repo with different tutorials: https://github.com/yunjey/pytorch-tutorial*

*4). Facebook AI Research (main contributor of PyTorch) website: https://facebook.ai/developers/tools*

# Что почитать:
- [Regularized Logistic Regression is Strictly Convex](http://www.qwone.com/~jason/writing/convexLR.pdf)
- [SAGA: A Fast Incremental Gradient Method With Support for Non-Strongly Convex Composite Objectives
](https://www.di.ens.fr/~fbach/Defazio_NIPS2014.pdf)
- [Лекции Евгения Соколова](https://github.com/esokolov/ml-course-hse/tree/master/2018-fall/lecture-notes)